# Evaluation

For evaluation distance scores have been utilized. The following metrics are calculated:

- SSIM (Structural Similarity Index Measure)
The Structural Similarity Index (SSIM) quantifies the similarity between two images based on luminance, contrast, and structure, aiming to measure perceived changes in image quality. Lies between 0 and 1 whereby 1 is identity.

- Wasserstein distance
The Wasserstein distance measures the minimum amount of work required to transform one probability distribution into another, providing a metric for comparing the dissimilarity between two distributions. A positive number representing the distance where lower is better. For our dataset, everything under 25 is good and under 10 is considered great.

- FID (Fréchet Inception Distance)
The Fréchet Inception Distance (FID) calculates the similarity between two datasets of images by comparing statistics extracted from their feature representations generated by a pre-trained Inception-v3 network. A positive number representing the distance where lower is better. Also here, everything under 10 is considered great.

In [3]:
import os
import torch
import pandas as pd
from math import log2
from tqdm import tqdm

from model.Generator import Generator

from functions.quality_metrics import ssim_metric, waterstein_distance_metric, fid_metric
from functions.example_generator import generate_examples

from utils.get_device import get_device

Reference dataset declaration.

In [4]:
real_datasets = {"2D_assets": "./dataset/2D_assets/transformed/", 
                 "bulbapedia": "./dataset/bulbapedia/transformed/",
                 "all_assets_and_bulbapedia": "./dataset/all_assets_and_bulbapedia/transformed/",
                 "all_assets": "./dataset/all_assets/transformed/"
                 }

Datasets / Data to compare / measure the quality for. 256 models have been ignored for fair comparison.

In [5]:
generators_paths = [
                    "./outputs/6_test_6_bulbapedia_265x265/generator_128.pth",
                    "./outputs/23_first_try_bulbapedia_23_bulbapedia_256x256_e50/generator_128.pth",
                    "./outputs/29_giga_dataset_29_all_assets_and_bulbapedia_256x256_e50/generator_64_40.pth",
                    "./outputs/31_fine_tune_23_with_giga_dataset_31_all_assets_and_bulbapedia_128x128_e100/generator_128_150.pth",
                    "./outputs/31_fine_tune_23_with_giga_dataset_31_all_assets_and_bulbapedia_128x128_e100/generator_128_130.pth",
                    "./outputs/33_fine_tune_23_with_clean_dataset_33_2D_assets_128x128_e100/generator_128_150.pth",
                    "./outputs/1_test_1_2D_assets_128x128_e30/generator_128.pth",
                    "./outputs/36_fine_tune_1_with_clean_dataset_36_2D_assets_128x128_e100/generator_128_150.pth"
                    ]

In [6]:
df = pd.DataFrame(columns=["generator", "dataset", "ssim", "waterstein", "fid", "path"])
backup_name = "evaluation"

Calculate SSIM, Wasserstein Distance, and FID.

In [7]:
base_path = "evaluation"
z_dim = 256
in_channels = 256
img_channels = 3
device = get_device()
amount_examples = 50
sample = 500

with tqdm(total=len(generators_paths)*len(real_datasets)) as pbar:
    for path in tqdm(generators_paths):
        gen = Generator(
            256, 256, img_channels=img_channels).to(device)
        gen.load_state_dict(torch.load(path))
        generator_size = int(path.split("/")[-1].split("_")[1].split(".")[0])
        step = int(log2(generator_size / 4))
        generator_name = path.split("/")[-2]
        generations_path = os.path.join(base_path, generator_name)
        os.makedirs(generations_path, exist_ok=True)
        generate_examples(gen, step, z_dim, device=device, path=generations_path, n=amount_examples)
        generations_path = os.path.join(generations_path, f"step{step}")
        
        for ds_name, ds_path in real_datasets.items():
            ssim = ssim_metric(ds_path, generations_path, sample=sample)
            waterstein = waterstein_distance_metric(ds_path, generations_path, sample=sample)
            fid = fid_metric(ds_path, generations_path, device=device)
            df.loc[len(df)] = [generator_name, ds_name, ssim, waterstein, fid, generations_path]
            df.to_csv(os.path.join(base_path,f"{backup_name}.csv"), index=False)
            pbar.update()

  0%|          | 0/8 [00:00<?, ?it/s]

<module 'pytorch_fid.fid_score' from '/root/.local/lib/python3.10/site-packages/pytorch_fid/fid_score.py'>








































































100%|██████████| 69/69 [00:15<00:00,  4.33it/s]


 12%|█▎        | 1/8 [02:03<14:24, 123.47s/it]

<module 'pytorch_fid.fid_score' from '/root/.local/lib/python3.10/site-packages/pytorch_fid/fid_score.py'>

































































































































































100%|██████████| 158/158 [00:31<00:00,  5.09it/s]


 25%|██▌       | 2/8 [04:49<14:50, 148.43s/it]

...
































































































































































100%|██████████| 158/158 [00:34<00:00,  4.54it/s]


 75%|███████▌  | 6/8 [18:10<05:54, 177.41s/it]

<module 'pytorch_fid.fid_score' from '/root/.local/lib/python3.10/site-packages/pytorch_fid/fid_score.py'>







































































































































































































































































































































































100%|██████████| 356/356 [01:13<00:00,  4.86it/s]


 88%|████████▊ | 7/8 [22:58<03:33, 213.83s/it]

<module 'pytorch_fid.fid_score' from '/root/.local/lib/python3.10/site-packages/pytorch_fid/fid_score.py'>













































































































































































































































100%|██████████| 235/235 [00:48<00:00,  4.85it/s]


100%|██████████| 8/8 [26:29<00:00, 198.70s/it]
